<a href="https://colab.research.google.com/github/JosephBL27/Theta-Decay/blob/main/Theta_Decay_Black_Scholes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================================================
# COMPLETE SELF-CONTAINED CODE CELL FOR GOOGLE COLAB
# =========================================================

import sys
import os
import json
import math
from datetime import datetime as dt

##############################
# 1) DUMMY BROKER & DEPENDENCIES
##############################

# -------------------------------------------------
# Dummy Implementation for google-cloud-secretmanager
# -------------------------------------------------
try:
    from google.cloud import secretmanager
except ImportError:
    class DummyPayload:
        def __init__(self):
            self.data = b'{"api_key": "dummy_api_key", "app_secret": "dummy_app_secret"}'
    class DummyResponse:
        def __init__(self):
            self.payload = DummyPayload()
    class DummySecretManagerServiceClient:
        def access_secret_version(self, request):
            return DummyResponse()
    class DummySecretManagerModule:
        SecretManagerServiceClient = DummySecretManagerServiceClient
    secretmanager = DummySecretManagerModule

# -------------------------------------------------
# Dummy Implementation for httpx (only .codes.OK)
# -------------------------------------------------
try:
    import httpx
except ImportError:
    class DummyCodes:
        OK = 200
    class DummyHTTPX:
        codes = DummyCodes
    httpx = DummyHTTPX

# -------------------------------------------------
# Dummy Implementation for the "schwab" Library
# -------------------------------------------------
class DummyClient:
    """
    Simulates a brokerage client object.
    """
    def __init__(self):
        self.accounts = [{"accountNumber": "123456", "hashValue": "dummy_hash"}]

    def get_account_numbers(self):
        class DummyResponse:
            def json(inner_self):
                return self.accounts
        return DummyResponse()

    def place_order(self, acct_hash, order_builder):
        class DummyOrder:
            status_code = 201
            order_id = "dummy_order_id"
        return DummyOrder()

    def get_order(self, order_id, acct_hash):
        class DummyOrderInfo:
            def json(inner_self):
                return {
                    "order_id": order_id,
                    "status": "FILLED",
                    "orderLegCollection": [{"instrument": {"symbol": "DUMMY"}}],
                    "orderActivityCollection": [{"executionLegs": [{"price": 1.23}]}]
                }
        return DummyOrderInfo()

    def cancel_order(self, order_id, acct_hash):
        class DummyCancellation:
            status_code = 200
        return DummyCancellation()

    def get_option_chain(self, symbol, **attrs):
        """
        Returns dummy option chain data: PUTs or CALLs with a few strikes
        and random implied volatilities. The user can set 'contract_type' to
        'PUT' or 'CALL' in 'attrs'.
        """
        dummy_data = {
            "underlyingPrice": 100.0,
            "callExpDateMap": {
                "2024-10-01": {
                    "100.0": [{"optionType": "CALL", "impliedVolatility": 0.22}],
                    "105.0": [{"optionType": "CALL", "impliedVolatility": 0.27}]
                },
                "2024-11-01": {
                    "110.0": [{"optionType": "CALL", "impliedVolatility": 0.25}],
                }
            },
            "putExpDateMap": {
                "2024-10-01": {
                    "95.0":  [{"optionType": "PUT", "impliedVolatility": 0.21}],
                    "90.0":  [{"optionType": "PUT", "impliedVolatility": 0.28}]
                },
                "2024-11-01": {
                    "85.0":  [{"optionType": "PUT", "impliedVolatility": 0.26}],
                }
            }
        }
        class DummyResponse:
            def json(self):
                return dummy_data
        return DummyResponse()

class DummyAuth:
    """
    Simulates various OAuth flows.
    """
    @staticmethod
    def easy_client(**args):
        return DummyClient()
    @staticmethod
    def client_from_manual_flow(**args):
        return DummyClient()
    @staticmethod
    def client_from_login_flow(**args):
        return DummyClient()

class DummyOptionSymbol:
    """
    Creates a dummy option symbol for placing orders.
    """
    def __init__(self, symbol, expiration, option_type, strike):
        self.symbol = symbol
        self.expiration = expiration
        self.option_type = option_type
        self.strike = strike
    def build(self):
        return f"{self.symbol}_{self.expiration.strftime('%Y%m%d')}_{self.option_type}_{self.strike}"

class DummyOrdersOptions:
    OptionSymbol = DummyOptionSymbol

    @staticmethod
    def option_buy_to_open_limit(option_symbol, quantity, limit_price):
        return {
            "order_type": "option_buy_to_open_limit",
            "symbol": option_symbol,
            "quantity": quantity,
            "limit_price": limit_price
        }

    @staticmethod
    def option_sell_to_open_limit(option_symbol, quantity, limit_price):
        return {
            "order_type": "option_sell_to_open_limit",
            "symbol": option_symbol,
            "quantity": quantity,
            "limit_price": limit_price
        }

class DummyOrdersEquities:
    @staticmethod
    def equity_buy_market(symbol, shares):
        return {"order_type": "equity_buy_market", "symbol": symbol, "shares": shares}
    @staticmethod
    def equity_sell_market(symbol, shares):
        return {"order_type": "equity_sell_market", "symbol": symbol, "shares": shares}
    @staticmethod
    def equity_buy_limit(symbol, shares, price):
        return {
            "order_type": "equity_buy_limit",
            "symbol": symbol,
            "shares": shares,
            "price": price
        }
    @staticmethod
    def equity_sell_limit(symbol, shares, price):
        return {
            "order_type": "equity_sell_limit",
            "symbol": symbol,
            "shares": shares,
            "price": price
        }

class DummyUtils:
    class Utils:
        def __init__(self, client, acct_hash):
            self.client = client
            self.acct_hash = acct_hash
        def extract_order_id(self, order):
            return order.order_id

class DummyOptions:
    class ContractType:
        CALL = "CALL"
        PUT = "PUT"

class DummySchwabClientModule:
    Options = DummyOptions

# Construct the top-level "schwab" module with submodules
class DummySchwab:
    auth = DummyAuth
    orders = type("Orders", (), {
        "options": DummyOrdersOptions,
        "equities": DummyOrdersEquities
    })
    utils = DummyUtils
    client = DummySchwabClientModule

# Insert the dummy schwab module into sys.modules so that "import schwab" works.
sys.modules["schwab"] = DummySchwab
import schwab


##############################
# 2) TRADING HELPER LIBRARY
##############################

account_number = ''
secret_version = ''
client = None
acct_hash = None

def init(secret_version_name=None, method='easy', account_number=None):
    auth(secret_version_name, method)
    set_account(account_number)

def auth(secret_version_name=None, method='easy'):
    global client, secret_version
    if secret_version_name is None:
        secret_version_name = 'projects/dummy_project/secrets/dummy_secret/versions/1'
    # Use dummy secret manager
    secret_manager_client = secretmanager.SecretManagerServiceClient()
    response = secret_manager_client.access_secret_version(request={"name": secret_version_name})
    payload = json.loads(response.payload.data.decode("UTF-8"))
    secret_version = secret_version_name
    args = {
        'api_key': payload['api_key'],
        'app_secret': payload['app_secret'],
        'callback_url': 'https://127.0.0.1:8182',
        'token_path': 'credentials/schwab_token.json'
    }
    # Pick an auth flow
    if method == 'easy':
        c = schwab.auth.easy_client(**args)
    elif method == 'login':
        c = schwab.auth.client_from_login_flow(**args)
    else:
        c = schwab.auth.client_from_manual_flow(**args)
    global client
    client = c

def set_account(acct_number):
    global accounts, acct_hash, account_number
    if 'accounts' not in globals():
        accounts = client.get_account_numbers().json()
    found = False
    for a in accounts:
        if a['accountNumber'] == str(acct_number):
            acct_hash = a['hashValue']
            account_number = acct_number
            found = True
            break
    if not found:
        acct_hash = accounts[0]['hashValue']
        account_number = accounts[0]['accountNumber']

def place_order(order_builder):
    order = client.place_order(acct_hash, order_builder)
    assert order.status_code == 201
    return schwab.utils.Utils(client, acct_hash).extract_order_id(order)

def option_buy_to_open_limit(symbol, expiration, option_type, strike_price, quantity, limit_price):
    if isinstance(expiration, str):
        expiration = dt.strptime(expiration, '%Y-%m-%d')
    option_symbol = schwab.orders.options.OptionSymbol(symbol, expiration, option_type, str(strike_price)).build()
    builder = schwab.orders.options.option_buy_to_open_limit(option_symbol, quantity, str(limit_price))
    return place_order(builder)

def option_sell_to_open_limit(symbol, expiration, option_type, strike_price, quantity, limit_price):
    if isinstance(expiration, str):
        expiration = dt.strptime(expiration, '%Y-%m-%d')
    option_symbol = schwab.orders.options.OptionSymbol(symbol, expiration, option_type, str(strike_price)).build()
    builder = schwab.orders.options.option_sell_to_open_limit(option_symbol, quantity, str(limit_price))
    return place_order(builder)

def equity_buy_market(symbol, shares):
    builder = schwab.orders.equities.equity_buy_market(symbol, shares)
    return place_order(builder)

def equity_sell_market(symbol, shares):
    builder = schwab.orders.equities.equity_sell_market(symbol, shares)
    return place_order(builder)

def equity_buy_limit(symbol, shares, price):
    builder = schwab.orders.equities.equity_buy_limit(symbol, shares, price)
    return place_order(builder)

def equity_sell_limit(symbol, shares, price):
    builder = schwab.orders.equities.equity_sell_limit(symbol, shares, price)
    return place_order(builder)

def get_order(order_id):
    return client.get_order(order_id, acct_hash).json()

def cancel_order(order_id):
    cancellation = client.cancel_order(order_id, acct_hash)
    return cancellation.status_code == httpx.codes.OK

##############################
# 3) MATH & ADVANCED STRATEGIES
##############################

from scipy.stats import norm

def black_scholes_price(S, K, T, r, sigma, option_type='CALL'):
    """
    Compute the Black-Scholes option price (no dividends).
    """
    # T is in years, r is annual, sigma is annual vol
    if T <= 0:
        return max(0.0, S - K) if option_type.upper() == 'CALL' else max(0.0, K - S)

    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    if option_type.upper() == 'CALL':
        price = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    else:
        price = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return price

def black_scholes_delta(S, K, T, r, sigma, option_type='CALL'):
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    if option_type.upper() == 'CALL':
        return norm.cdf(d1)
    else:
        return norm.cdf(d1) - 1

def black_scholes_theta(S, K, T, r, sigma, option_type='CALL'):
    """
    Computes the daily Theta (time decay).
    """
    if T <= 0:
        return 0.0
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    pdf_d1 = norm.pdf(d1)
    if option_type.upper() == 'CALL':
        # standard annual Theta
        theta_annual = (-S * pdf_d1 * sigma / (2*math.sqrt(T))) - (r*K*math.exp(-r*T)*norm.cdf(d2))
    else:
        theta_annual = (-S * pdf_d1 * sigma / (2*math.sqrt(T))) + (r*K*math.exp(-r*T)*norm.cdf(-d2))
    # Convert to daily
    return theta_annual / 365.0

def approximate_dtheta_dtime(S, K, T, r, sigma, option_type='CALL', eps=1/365):
    """
    Approximates the derivative of Theta with respect to time.
    We'll do a finite difference:
       dTheta/dT ~ [Theta(S,K,T - eps) - Theta(S,K,T)] / eps

    Because as we move T->T - eps, there's 'eps' days fewer until expiration,
    so the sign indicates how quickly theta itself is changing w.r.t. T.
    """
    # current Theta
    theta_now = black_scholes_theta(S, K, T, r, sigma, option_type)
    # Theta if time is shortened by eps
    # (meaning T' = T - eps => T' is "eps" days closer to expiration)
    T_short = max(0.0, T - eps)
    theta_future = black_scholes_theta(S, K, T_short, r, sigma, option_type)
    # finite difference
    return (theta_future - theta_now) / eps

def screen_option_chain(option_chain, underlying_price, risk_free_rate=0.01, target_theta_max=None, delta_range=None):
    """
    Screens the option chain for options meeting specified Greek criteria.
    """
    filtered_options = []
    now = dt.now()

    for exp_date_str, strikes_dict in option_chain.items():
        # parse expiration date
        try:
            exp_date = dt.strptime(exp_date_str, '%Y-%m-%d')
        except:
            continue
        T = (exp_date - now).days / 365.0
        if T <= 0:
            continue

        for strike_str, option_list in strikes_dict.items():
            try:
                strike = float(strike_str)
            except:
                continue
            for opt_data in option_list:
                opt_type = opt_data.get('optionType', 'CALL')
                sigma = opt_data.get('impliedVolatility', 0.2)

                # compute Delta
                delta_val = black_scholes_delta(underlying_price, strike, T, risk_free_rate, sigma, opt_type)
                if delta_range is not None:
                    if not (delta_range[0] <= delta_val <= delta_range[1]):
                        continue

                # compute daily Theta
                theta_val = black_scholes_theta(underlying_price, strike, T, risk_free_rate, sigma, opt_type)
                if target_theta_max is not None:
                    if abs(theta_val) > target_theta_max:
                        continue

                # for demonstration, compute approximate derivative dTheta/dT
                dtheta_dtime = approximate_dtheta_dtime(underlying_price, strike, T, risk_free_rate, sigma, opt_type)

                # add computed fields
                opt_data['computed_delta'] = delta_val
                opt_data['computed_theta'] = theta_val
                opt_data['computed_dtheta_dtime'] = dtheta_dtime
                opt_data['time_to_expiration'] = T
                opt_data['strike'] = strike
                opt_data['expiration'] = exp_date_str
                filtered_options.append(opt_data)
    return filtered_options

##############################
# 4) INTRICATE AUTOMATED TRADING BOT
##############################

def pick_best_option_by_advanced_criteria(options_list):
    """
    Decide which option from 'options_list' is 'best' by a custom metric.
    We'll combine:
      - Low absolute theta (time decay)
      - Negative dTheta/dT (meaning theta is decreasing as time approaches,
        so time decay might slow down as we get closer)
      - Higher implied volatility if we want potential bigger premium
    For demonstration, define a 'score' for each option:

       score = -abs(theta) + (-dTheta_dtime)*0.5 + (sigma*0.1)

    The rationale:
      - We want a small abs(theta) => less daily time decay for a buyer
      - We want negative dTheta/dtime => indicates that time decay might
        ease as we approach expiry
      - We want some positive weighting for sigma => higher IV might yield
        bigger moves but also cost more. This is purely a demonstration.
    """
    best_opt = None
    best_score = -999999

    for opt_data in options_list:
        theta_val = opt_data.get('computed_theta', 0.0)
        dtheta_dtime = opt_data.get('computed_dtheta_dtime', 0.0)
        sigma = opt_data.get('impliedVolatility', 0.2)

        score = -abs(theta_val) + (-dtheta_dtime)*0.5 + (sigma*0.1)

        if score > best_score:
            best_score = score
            best_opt = opt_data

    return best_opt, best_score

def auto_trade_bot():
    """
    This function simulates a cyclical trading routine that:
    1. Retrieves a dummy option chain.
    2. Screens for put or call options with certain criteria (Delta range, max Theta).
    3. Picks the 'best' option by a custom advanced metric.
    4. Places a dummy limit order to buy that option if it meets thresholds.
    5. (Pretends to) check fill status, etc.

    In a real scenario, you'd run this repeatedly or schedule it,
    but here we just show a single pass.
    """
    # 1) Retrieve dummy option chain
    symbol = "DUMMY"
    # random choice of 'CALL' or 'PUT'. Let's do CALL for demonstration
    option_type = "CALL"
    attrs = {
        'contract_type': getattr(schwab.client.Options.ContractType, option_type.upper()),
        'include_underlying_quote': True,
        # We'll look at a wide date range
        'from_date': dt(2024, 10, 1),
        'to_date':   dt(2024, 12, 31)
    }
    fetched_data = client.get_option_chain(symbol, **attrs).json()

    if option_type.upper() == 'CALL':
        option_chain = fetched_data.get('callExpDateMap', {})
        delta_range = (0.2, 0.9)  # broad range for calls
    else:
        option_chain = fetched_data.get('putExpDateMap', {})
        delta_range = (-0.9, -0.2)

    # 2) Screen for moderate daily Theta and suitable Delta
    screened = screen_option_chain(
        option_chain,
        underlying_price = fetched_data.get('underlyingPrice', 100),
        risk_free_rate   = 0.01,
        target_theta_max = 0.10,  # let daily time decay up to 10 cents
        delta_range      = delta_range
    )

    if not screened:
        print("No options matched the screening criteria.")
        return

    # 3) Pick the best option by advanced metric
    best_opt, score = pick_best_option_by_advanced_criteria(screened)

    if best_opt is None:
        print("No best option found after scoring.")
        return

    # 4) Place a dummy limit order to buy the 'best' option
    # We can guess a limit price by using the Black-Scholes price
    # We'll add a small negative offset to simulate a 'desired better fill'.
    S = fetched_data.get('underlyingPrice', 100)
    K = best_opt['strike']
    T = best_opt['time_to_expiration']
    sigma = best_opt['impliedVolatility']
    r = 0.01

    bs_price = black_scholes_price(S, K, T, r, sigma, best_opt['optionType'])
    desired_limit = max(0.0, bs_price - 0.1)  # a small offset

    print(f"Auto-trading the best {best_opt['optionType']} option found:")
    print(f"  Expiration: {best_opt['expiration']}, Strike: {K}")
    print(f"  Delta: {best_opt['computed_delta']:.3f}, Theta: {best_opt['computed_theta']:.3f}, "
          f"dTheta/dT: {best_opt['computed_dtheta_dtime']:.6f}, IV: {sigma}")
    print(f"  Black-Scholes Price: {bs_price:.2f}, Limit: {desired_limit:.2f} (score: {score:.3f})")

    # We'll do a buy-to-open limit order with quantity=1
    order_id = option_buy_to_open_limit(
        symbol      = symbol,
        expiration  = best_opt['expiration'],
        option_type = best_opt['optionType'],
        strike_price= K,
        quantity    = 1,
        limit_price = desired_limit
    )
    print(f"Placed dummy order with ID = {order_id}")

    # 5) (Pretend to) poll for fill
    status_data = get_order(order_id)
    print(f"Order status: {status_data['status']} (dummy fill).")
    if status_data['status'] != 'FILLED':
        # Cancel if not filled
        canceled = cancel_order(order_id)
        print("Order canceled:", canceled)
    else:
        print("Order filled! (dummy simulation).")

##############################
# 5) EXAMPLE MAIN LOGIC
##############################
if __name__ == '__main__':
    # Initialize the dummy environment
    init(account_number="123456")  # dummy
    print("Running advanced, dummy HFT-style option trading bot...")

    # Run the auto trade bot logic once
    auto_trade_bot()

    # In a real system, you'd schedule or repeatedly call auto_trade_bot()
    # over time, possibly with different symbols or updated parameters.


Running advanced, dummy HFT-style option trading bot...
No options matched the screening criteria.
